# Import modules

In [2]:
%matplotlib 
import matplotlib.pyplot as plt
import numpy as np
from scipy.linalg import sqrtm, block_diag
from math import pi
from qutip import *
import sympy
import functools, operator
from collections import OrderedDict
import itertools

Using matplotlib backend: TkAgg


# Two transmon + bus 
Following Magesan's paper

## Function define

In [156]:
def hamiltonian_eff_H0(w1,w2,J,alpha1,alpha2,N):
    """ 
    Two-transmon effective Hamiltonian with zero excitation in bus. 
    Eq. 2.12, Magesan
    w1, w2:  angular freq
    """
    b = destroy(N)
        
    I = qeye(N)

    # Eq. 2.21, Magesan
    H1 = w1*tensor(b.dag()*b, I) + alpha1/2.0*tensor(b.dag()*b.dag()*b*b, I)
    H2 = w2*tensor(I, b.dag()*b) + alpha2/2.0*tensor(I, b.dag()*b.dag()*b*b)
    H12 = J*(tensor(b.dag(), b) + tensor(b, b.dag())) 
    
    H_eff = H1 + H2 + H12
    
    return H_eff

def hamiltonian_eff_Hd(wd, Omega_X, Omega_Y, t, N):
    """ 
    CR drive Hamiltonian. Eq.2.14.
    All arguments are single number, not an array
    wd: angular freq
    """
    b = destroy(N)  # qubit 2 is target qubit
#     Hd = (Omega_X * np.cos(wd*t) + Omega_Y * np.sin(wd*t)) * tensor(qeye(N),b + b.dag())
    Hd = (Omega_X * np.cos(wd*t) + Omega_Y * np.sin(wd*t)) * tensor(b + b.dag(),qeye(N))
    
    return Hd

def ZZ_computed(J, alpha1, alpha2, Delta):
    """ Eq.4.5 in Magesan"""
    return -J**2*(1/(Delta + alpha1) + 1/(-Delta + alpha2)) * 2

def sorted_diag_op(eigen):
    """
    Diagonalizing matrix by the order of eigenvector
    
    Args:
        eigen : output of eigenstates() method in qutip
       
    Return: a new diagonalizing operator Qobj
    """
       
    index_list = [np.argmax(np.absolute(vec)) for vec in eigen[1]]
    print(index_list)
    X_array = np.column_stack([eigen[1][index_list.index(i)] for i in range(eigen[1].size)])
    X = Qobj(X_array)
    
    return X

In [3]:
# parameters for IBM qubit
w1, w2 = 2*pi*5, 2*pi*5.2 # in GHz
J = 0.005*2*pi
alpha1, alpha2 = -0.35*2*pi, -0.35*2*pi # in GHz
N=5

H_eff = hamiltonian_eff_H0(w1,w2,J,alpha1,alpha2,N)
eigen = H_eff.eigenstates()
ZZ = eigen[0][5] - eigen[0][1] - eigen[0][2]  # in GHz

sympy.SparseMatrix(np.round((H_eff)[:, 0:13],2))
print(f'ZZ = {ZZ*1e3/(2*pi):.3f} (MHz)')
print(f'ZZ_by_formula = {ZZ_computed(J,alpha1,alpha2,w1-w2)*1e3/(2*pi):.3f} (MHz)')

ZZ = 0.423 (MHz)
ZZ_by_formula = 0.424 (MHz)


In [4]:
eigen[0]

array([  0.        ,  31.41514163,  32.67334851,  60.63216698,
        63.14392431,  64.09114939,  87.64991143,  91.4194034 ,
        93.30269448,  94.56601264, 112.46851434, 117.49482602,
       120.3217872 , 122.83078145, 123.78669517, 145.14018436,
       148.90897676, 150.79083124, 152.06261183, 175.61160088,
       178.11596075, 179.08655243, 203.87692006, 205.15844344,
       229.96458224])

## CR Driven: two-level qubit case

In [203]:
w1, w2 = 2*pi*5, 2*pi*5.2 # in GHz
J = 0.01*2*pi
alpha1, alpha2 = -0.35*2*pi, -0.35*2*pi
N =2
Omega_X, Omega_Y = 0.02*2*pi,0*2*pi

b1 = destroy(N)
b2 = destroy(N)

HA = w2*(tensor(b1.dag()*b1, qeye(N)) + tensor(qeye(N), b2.dag()*b2))
R = lambda t: (-1j*HA*t).expm()

H0 = hamiltonian_eff_H0(w1,w2,J,alpha1,alpha2,N)
Hd = lambda t:hamiltonian_eff_Hd(w2, Omega_X, Omega_Y, t, N)

HR = lambda t: R(t).dag() * (H0 + Hd(t)) * R(t) - HA

In [204]:
t_list = np.linspace(0, 2, 401)  # in ns
HR_list = [HR(t) for t in t_list]
H_R = functools.reduce(operator.add, HR_list) / len(HR_list)

In [205]:
H_R.conj().trans()
H_R

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[ 0.        +0.j          0.        +0.j          0.06248237-0.00025391j
   0.        +0.j        ]
 [ 0.        +0.j          0.        +0.j          0.06283185+0.j
   0.06248237-0.00025391j]
 [ 0.06248237+0.00025391j  0.06283185+0.j         -1.25663706+0.j
   0.        +0.j        ]
 [ 0.        +0.j          0.06248237+0.00025391j  0.        +0.j
  -1.25663706+0.j        ]]

In [17]:
avg_list = []
for N in range(2,401):
    temp = (functools.reduce(operator.add,[HR_list[i] for i in range(N)]) / N)
    temp = np.real(temp[2,0])
    avg_list.append(temp)
    
plt.plot(avg_list)
# avg_list

In [209]:
# Least action
# find X
X_array = np.column_stack( (eigen[1][0],eigen[1][1],eigen[1][2],eigen[1][3]))
X = Qobj(X_array)

# find X_BD and XP
A, B = X[0:2,0:2], X[2:4, 2:4]
X_BD_array = block_diag(A,B)
X_BD = Qobj(X_BD_array)
XP = X_BD * X_BD.conj().trans()

# find T
T = X * X_BD.conj().trans() * XP.sqrtm().inv()

# find H_R_BD
T.dims = H_R.dims
H_R_BD =  T.conj().trans() * H_R * T
H_R_BD

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[-1.2597286 +0.00000000e+00j -0.00310116+1.26023328e-05j
   0.        +0.00000000e+00j  0.        +0.00000000e+00j]
 [-0.00310116-1.26023328e-05j -1.26284711+0.00000000e+00j
   0.        +0.00000000e+00j  0.        +0.00000000e+00j]
 [ 0.        +0.00000000e+00j  0.        +0.00000000e+00j
   0.00621004+0.00000000e+00j  0.00310116-1.26023328e-05j]
 [ 0.        +0.00000000e+00j  0.        +0.00000000e+00j
   0.00310116+1.26023328e-05j  0.00309153+0.00000000e+00j]]

## CR Driven: three-level case
1. get block-diagonal full H_RWA Hamiltonina, then take computational subspace
2. To fine Pauli coefficients: ZX,ZZ,ZI,IX,and IZ
3. Main question: how to compute H_RWA. How can we ignore other than e^{-iwt} or e^{iwt}.

In [234]:
def get_Pauli(Omega):
    w1, w2 = 2*pi*5.164, 2*pi*5.292 # in GHz
    J = 0.075*2*pi
    alpha1, alpha2 = 0.6*2*pi, -0.33*2*pi
    # Omega = 0.001*2*pi
    N =3

    w1_b = [0, w1, 2*w1+alpha1]
    w2_b = [0, w2, 2*w2+alpha2]

    b = destroy(N)

    # H_eff = hamiltonian_eff_H0(w1,w2,J,alpha1,alpha2,N)

    def get_possible_states(n):
        """
        Get a list of tuples of possible states, given n
        Ex) for n=2, output is [(0,0),(0,1),(1,0),(0,2),(1,1),(2,0)]
        Args: 
            n: integer such that n1+n2<=n for |n1,n2> state
        Return:
            List of tuples that satisfy n1+n2<=n where n1 and n2 are 0,1,2,3,....
        """
        def get_possible_sum(n):
            """
             [(0,1)]
            """
            result = []
            for i in range(n+1):
                result.append((i,n-i))
            return result

        possible_list = []
        for i in range(n + 1):
            possible_list += get_possible_sum(i)

        return possible_list
    def kronecker_delta(i,j):
        return (1 if i==j else 0)

    # Note
    # b1[i,j] = <i|b1|j>, where |i> = {|00>, |01>, |10>, |11>, ... |30>}
    # b1|n1_i,n2_i> = sqrt(n1_i)|n1_i-1,n2_i> where |i> = |n1_i, n2_i> and |j> = |n1_j, n2_j>
    # b2|n1_i,n2_i> = sqrt(n2_i)|n1_i,n2_i-1> where |i> = |n1_i, n2_i> and |j> = |n1_j, n2_j>
    # V = sqrt((n1+1)*(n2+1))*J_n1_n2*(|n1+1,n2><n1,n2+1|+|n1,n2+1><n1+1,n2|)

    states = [ prod for prod in itertools.product(range(3), range(3))]
    unsorted_state_energy_dict = { s: w1_b[s[0]] + w2_b[s[1]] for s in states}
    sorted_state_energy_dict = OrderedDict(sorted(unsorted_state_energy_dict.items(), key=lambda x: x[1]))

    sorted_energy = list(sorted_state_energy_dict.values())
    sorted_states = list(sorted_state_energy_dict.keys())
    # [(0,0),(0,1),(1,0),(1,1),(0,2),(2,0),(0,3),(1,2),(2,1),(3,0)]
    sorted_states= [(0, 0), (0, 1), (1, 0),  (1, 1), (0, 2), (2, 0),(1, 2), (2, 1), (2, 2)]
    # sorted_states= [(0, 0), (1, 0), (0, 1), (0, 2), (1, 1),  (2, 0),(1, 2), (2, 1), (2, 2)]
    sorted_energy = [ w1_b[state[0]]+w2_b[state[1]] for state in sorted_states]

    n = len(sorted_state_energy_dict)
    b1_array, b2_array, V0_array = np.zeros((n,n)), np.zeros((n,n)), np.zeros((n,n))

    # redefind b1, b2 with basis of sorted states
    for i, j in itertools.product(range(n), range(n)): # i, j = basis index
        n1_i, n2_i  = sorted_states[i][0], sorted_states[i][1]
        n1_j, n2_j = sorted_states[j][0], sorted_states[j][1]    

        # b1_array[i,j] = <n1_i, n2_i|b1|n1_j, n2_j>
        b1_array[i, j] = np.sqrt(n1_j)* kronecker_delta(n1_i, n1_j-1) * kronecker_delta(n2_i, n2_j)
        b2_array[i, j] = np.sqrt(n2_j)* kronecker_delta(n2_i, n2_j-1) * kronecker_delta(n1_i, n1_j) 

    b1 = Qobj(b1_array)
    b2 = Qobj(b2_array)

    # H0 = Qobj(np.diag(sorted_energy))    
    H0 = w1*b1.dag()*b1 + alpha1/2.0*b1.dag()*b1.dag()*b1*b1 + w2*b2.dag()*b2 \
        + alpha2/2.0*b2.dag()*b2.dag()*b2*b2    
    V0 = J*(b1.dag()*b2 + b1*b2.dag()) 
    H_eff = H0 + V0

    b1t = b1 + b1.dag()
    b2t = b2 + b2.dag()
    ntot = b1.dag()*b1 + b2.dag()*b2

    # digonalize H0: remove J
    eigen = H_eff.eigenstates()

    # find U
    # U_array = np.column_stack( [eigen[1][i] for i in range(eigen[1].size)])
    U_array = sorted_diag_op(eigen)
    U = Qobj(U_array)
    U.dims = H0.dims

    # digonalize H0
    H_eff_D = U.dag() * H_eff * U

    # get new Hd and HR
    wd = (H_eff_D[3,3] - H_eff_D[1,1] + H_eff_D[2,2])/2
    Hd = lambda t:Omega * np.cos(wd*t)*b1t
    Hd_D = lambda t: U.dag() * Hd(t) * U

    # To rotating frame
    HA = wd*(b1.dag()*b1 + b2.dag()*b2)
    R = lambda t: (-1j*HA*t).expm()
    HR = lambda t: R(t).dag() * (H_eff_D + Hd_D(t)) * R(t) - HA

    # average over 10 period
    t_list = np.linspace(0, 2, 401)  # in ns
    HR_list = [HR(t) for t in t_list]
    H_R = functools.reduce(operator.add,HR_list) / len(HR_list)
    H_R.tidyup(1e-3)

    # least action
    eigen = H_R.eigenstates()

    # find X
    # X_array = np.column_stack( [eigen[1][i] for i in range(eigen[1].size)])
    X_array = sorted_diag_op(eigen)
    X = Qobj(X_array)

    # find X_BD and XP
    A, B, C = X[0:2,0:2], X[2:4, 2:4], X[4:10, 4:10]
    X_BD_array = block_diag(A,B,C)
    X_BD = Qobj(X_BD_array)
    # X_BD = X_BD.tidyup(atol=1e-3)
    XP = X_BD * X_BD.dag()

    # find T
    T = X * X_BD.dag() * XP.sqrtm().inv()

    # find H_R_BD
    T.dims = H_R.dims
    H_R_BD =  T.dag() * H_R * T
    H_R_BD

    def from16to8(M):
        return np.array([M[0,0],M[1,0],M[0,1],M[1,1],M[2,2],M[3,2],M[2,3],M[3,3]]).reshape(8,1)

    # Get Pauli coeff.
    II = tensor(qeye(2), qeye(2))
    IX = tensor(qeye(2), sigmax())
    IY = tensor(qeye(2), sigmay())
    IZ = tensor(qeye(2), sigmaz())
    ZI = tensor(sigmaz(), qeye(2))
    ZY = tensor(sigmaz(), sigmay())
    ZX = tensor(sigmaz(), sigmax())
    ZZ = tensor(sigmaz(), sigmaz())

    a = np.column_stack([from16to8(II),from16to8(IX),from16to8(IY),from16to8(IZ)
                        ,from16to8(ZI),from16to8(ZX),from16to8(ZY),from16to8(ZZ)])

    b = from16to8(H_R_BD[0:4, 0:4])

    c = np.linalg.solve(a, b)
#     sympy.Matrix(np.round(c,6))/2/pi*1000
    # -63.9, -0.02, -0.00017,1.2e-5,127.9,0.073, 0.00017, -0.039
    return c/2/pi*1e3

# plot Pauli coeff. vs CR amplitude
Omega_list = np.linspace(1, 140, 11)*1e-3*2*pi
Pauli = get_Pauli(Omega_list[0])
for omega in Omega_list[1:]:
    Pauli = np.column_stack((Pauli, get_Pauli(omega)))

# plot
Pauli_label = ['II','IX','IY','IZ','ZI','ZX','ZY','ZZ']
fig, ax = plt.subplots(1,1, figsize=(10,8))
for i in range(Pauli.shape[0]):
    ax.plot(Omega_list, Pauli[i], label=Pauli_label[i], linewidth=3)
ax.set_xlabel('CR amplitude, Omega (MHz)', fontsize=18)
ax.set_ylabel('$Pauli Coefficient$ (MHz)', fontsize=18)
ax.tick_params(axis='x', labelsize=18)
ax.tick_params(axis='y', labelsize=18)
ax.grid('on')
ax.legend(fontsize=18)

[0, 2, 1, 4, 3, 5, 6, 7, 8]
[4, 6, 2, 0, 1, 3, 8, 5, 7]
[0, 2, 1, 4, 3, 5, 6, 7, 8]
[4, 6, 2, 0, 1, 3, 8, 5, 7]
[0, 2, 1, 4, 3, 5, 6, 7, 8]
[4, 6, 2, 0, 1, 3, 8, 5, 7]
[0, 2, 1, 4, 3, 5, 6, 7, 8]
[4, 2, 6, 0, 1, 3, 8, 5, 7]
[0, 2, 1, 4, 3, 5, 6, 7, 8]
[4, 2, 6, 0, 1, 3, 8, 5, 7]
[0, 2, 1, 4, 3, 5, 6, 7, 8]
[4, 2, 6, 0, 1, 3, 8, 5, 7]
[0, 2, 1, 4, 3, 5, 6, 7, 8]
[4, 2, 6, 0, 1, 3, 8, 5, 7]
[0, 2, 1, 4, 3, 5, 6, 7, 8]
[4, 2, 6, 0, 1, 3, 8, 5, 7]
[0, 2, 1, 4, 3, 5, 6, 7, 8]
[4, 2, 6, 0, 1, 3, 8, 5, 7]
[0, 2, 1, 4, 3, 5, 6, 7, 8]
[4, 2, 6, 0, 1, 3, 8, 5, 7]
[0, 2, 1, 4, 3, 5, 6, 7, 8]
[4, 2, 6, 0, 1, 3, 8, 5, 7]


/usr/lib/python3/dist-packages/numpy/core/_asarray.py:85: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)


In [233]:
H_eff/2/pi
# H0
# V0
# states
# X
sorted_states
sorted_energy
H_R_BD

Quantum object: dims = [[9], [9]], shape = (9, 9), type = oper, isherm = True
Qobj data =
[[ 1.04849882e-04  1.31191863e-03  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00]
 [ 1.31191863e-03  1.16188195e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00 -7.71489187e-02 -2.00595151e-03
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00 -2.00595151e-03  1.23903749e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  -1.16593468e-12]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  -4.85849920e-01  3.56026057e-07 -3.12738641e-03  1.28651348e-03
  -3.79719397e-12]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   3.56026057e-07  4.19803445e+00 -1.77055897e-09 -1.90120157e-10
  -4.72665556e-12]
 [ 0

In [229]:
H0_D
U
H_eff_D
X_BD
X

Quantum object: dims = [[9], [9]], shape = (9, 9), type = oper, isherm = False
Qobj data =
[[ 9.99979052e-01  6.25446501e-03 -1.66628863e-03 -5.50652916e-07
   3.15388405e-05  1.11306541e-06 -1.42923849e-08 -9.10052454e-11
   0.00000000e+00]
 [ 6.25440574e-03 -9.99977738e-01  1.03017660e-05  1.33055301e-10
   2.13001200e-03  9.31908273e-04  2.10642974e-12 -5.12339747e-07
   0.00000000e+00]
 [-1.66647290e-03 -4.77484704e-06 -9.99972927e-01  5.21258895e-04
   7.09008988e-03  9.09497075e-04  1.46803489e-05  8.30076115e-07
  -6.82189320e-10]
 [-1.41987664e-06 -3.25442893e-09 -5.21439464e-04 -9.99561170e-01
   5.19460378e-06  2.04977014e-07 -2.96164989e-02  2.43302237e-04
  -1.01759242e-06]
 [ 3.30456242e-05 -2.12986656e-03 -7.09012741e-03 -1.21731204e-06
  -9.99971935e-01  2.26008228e-08 -3.27896680e-08  1.14950355e-03
   0.00000000e+00]
 [-5.42592737e-06  9.31885746e-04  9.09465745e-04 -2.69191988e-07
  -8.41083423e-06  9.99999152e-01 -7.41449304e-09  2.47790152e-10
   1.52097326e-12]
 [ 

In [197]:
avg_list = []
for N in range(2,401):
    temp = (functools.reduce(operator.add,[HR_list[i] for i in range(N)]) / N)
    temp = np.real(temp[0,1])
    avg_list.append(temp)
    

In [93]:
# plt.plot(avg_list)
f=np.fft.fft(avg_list)
plt.plot(np.absolute(f))
plt.yscale('log')

In [199]:
X.tidyup(1e-4)

Quantum object: dims = [[9], [9]], shape = (9, 9), type = oper, isherm = False
Qobj data =
[[ 7.31519078e-01 -6.81819791e-01  8.16590421e-04  9.30105165e-04
   0.00000000e+00  2.80424644e-04  0.00000000e+00  0.00000000e+00
   0.00000000e+00]
 [ 6.81685217e-01  7.31365731e-01  3.04653235e-03 -3.29706675e-03
   0.00000000e+00 -1.97301915e-02  0.00000000e+00  0.00000000e+00
   0.00000000e+00]
 [-8.59241377e-04  8.83754969e-04  7.41092998e-01  6.71380189e-01
   0.00000000e+00  5.31124334e-03  0.00000000e+00  2.86141310e-04
   0.00000000e+00]
 [ 2.95857400e-03  3.38110776e-03 -6.71246987e-01  7.40953031e-01
  -6.36373434e-04  0.00000000e+00  7.94402487e-04 -1.98710205e-02
   0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   7.97772657e-01  0.00000000e+00  6.02956315e-01 -1.54592346e-03
  -2.83164397e-04]
 [ 1.32518358e-02  1.46193858e-02 -3.85772426e-03 -3.65463032e-03
   0.00000000e+00  9.99785796e-01  0.00000000e+00  3.28421781e-03
   0.00000000e+00]
 [ 

In [202]:
test=Qobj(IX.data.reshape(16,1))
IX.data.reshape(16,1).todense()

matrix([[0.+0.j],
        [1.+0.j],
        [0.+0.j],
        [0.+0.j],
        [1.+0.j],
        [0.+0.j],
        [0.+0.j],
        [0.+0.j],
        [0.+0.j],
        [0.+0.j],
        [0.+0.j],
        [1.+0.j],
        [0.+0.j],
        [0.+0.j],
        [1.+0.j],
        [0.+0.j]])

# Transmon with a Duffing model

In [53]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from qutip import *
import sympy

In [6]:
# parameters for IBM qubit
w = 5 # in GHz
alpha = -0.35 # in GHz

N = 3
b = destroy(N)

H = w1*b.dag()*b + alpha/2.0*b.dag()*b*(b.dag()*b-qeye(N))
H1 = w1*b.dag()*b + alpha/2.0*b.dag()*b.dag()*b*b
Hd = b.dag() + b

In [9]:
H
# Hd

Quantum object: dims = [[3], [3]], shape = (3, 3), type = oper, isherm = True
Qobj data =
[[ 0.    0.    0.  ]
 [ 0.    5.3   0.  ]
 [ 0.    0.   10.25]]

# Find Pauli-coefficients of CSQF/transmon with CR
Ref: Xuexin's mathematica notebook
1. Build effective two-qubit Hamiltonian.
2. Diagonalize the two-qbuti Hamiltonian to get rid of coupling by either SW or numerical diagonalization. Dressed state.
3. Build driving Hamiltonian in the dressed state.
4. Move into rotating frame.
5. Block-diagonalize the total Hamiltonian by SW or least action.
6. Find Pauli-coefficients.

## Helper

In [235]:
def list_energies(f1,alpha1, beta1, f2, alpha2, beta2, sort=False):
    """
    Calculate energy levels. E00 = 0
    """
    numStates = 4 # 0,1,2,3 states
    state1 = [0, f1, f1+f1+alpha1,f1+f1+f1+alpha1+beta1]
    state2 = [0, f2, f2+f2+alpha2,f2+f2+f2+alpha2+beta2]
    
    result_dict = {}
    for prod in itertools.product(range(numStates), range(numStates)):
        temp = {prod:state1[prod[0]] + state2[prod[1]]}
        result_dict.update(temp)
    
    if sort:
        sorted_dict = OrderedDict(sorted(result_dict.items(), key=lambda x: x[1]))
        return sorted_dict
    else:
        return result_dict
    
list_energies(5.05, 0.6, 0.5, 5.25,-0.33, -0.37,sort=True)    
# list_energies(5.25,-0.33, -0.37, 5.05, -0.3, -0.33, sort=True)    

def sorted_diag_op(eigen):
    """
    Diagonalizing matrix by the order of eigenvector
    
    Args:
        eigen : output of eigenstates() method in qutip
       
    Return: a new diagonalizing operator Qobj
    """
       
    index_list = [np.argmax(np.absolute(vec)) for vec in eigen[1]]
    print(index_list)
    X_array = np.column_stack([eigen[1][index_list.index(i)] for i in range(eigen[1].size)])
    X = Qobj(X_array)
    
    return X

## Define qubit frequencies and J

In [265]:
wm = 5164       # CSFQ w01
del_m0 = 600   # first anharmonicity
del_m1 = 430  # second anharmon.
gdm = 80     # CSFQ-bus coupling

wr = 6292 # bus cavity

wt = 5292   # transmon
del_t0 = -329.1
del_t1 = -369.7
gdt = -80    # transmon-bus coupling

gmt = 0  # csfq-transmon direct g

# CSFQ, j=0,1,2
w1_d = lambda j: wm*j + del_m0/2*j*(j-1) if j<3 else wm*j + del_m0/2*j*(j-1) + del_m1-del_m0 # dressed freq
gamma1 = lambda j: j/(wr + w1_d(j-1) - w1_d(j))
w1_b = lambda j: w1_d(j) - gamma1(j) * gdm**2  # bare freq. j=1,2,3

# transmon, j=0,1,2
w2_d = lambda j: wt*j + del_t0/2*j*(j-1) if j<3 else wt*j + del_t0/2*j*(j-1) + del_t1-del_t0 # dressed freq
gamma2 = lambda j: j/(wr + w2_d(j-1) - w2_d(j))
w2_b = lambda j: w2_d(j) - gamma2(j) * gdt**2  # bare freq.

# J, {j1,j2}=0,1,2
J = lambda j1,j2 : gmt - gdm*gdt/2*(1/(wr+w1_d(j1)-w1_d(j1+1)) + 1/(wr+w2_d(j2)-w2_d(j2+1)) 
                                    + 1/(wr-w1_d(j1)+w1_d(j1+1)) + 1/(wr-w2_d(j2)+w2_d(j2+1))) 

In [266]:
J(0,1)
w1_d(1)-w1_d(0)
# w2_d(1)-w2_d(0)
j1=0
j2=1
- gdm*gdt/2*(1/(wr+w1_d(j1)-w1_d(j1+1)) + 1/(wr+w2_d(j2)-w2_d(j2+1)) 
                                    + 1/(wr-w1_d(j1)+w1_d(j1+1)) + 1/(wr-w2_d(j2)+w2_d(j2+1))) 
J(0,0)

6.592452135485313

## Build effective two-qubit Hamiltonian

In [325]:
def get_possible_states(n):
    """
    Get a list of tuples of possible states, given n
    Ex) for n=2, output is [(0,0),(0,1),(1,0),(0,2),(1,1),(2,0)]
    Args: 
        n: integer such that n1+n2<=n for |n1,n2> state
    Return:
        List of tuples that satisfy n1+n2<=n where n1 and n2 are 0,1,2,3,....
    """
    def get_possible_sum(n):
        """
         [(0,1)]
        """
        result = []
        for i in range(n+1):
            result.append((i,n-i))
        return result
    
    possible_list = []
    for i in range(n + 1):
        possible_list += get_possible_sum(i)
    
    return possible_list
def kronecker_delta(i,j):
    return (1 if i==j else 0)

# Note
# b1[i,j] = <i|b1|j>, where |i> = {|00>, |01>, |10>, |11>, ... |30>}
# b1|n1_i,n2_i> = sqrt(n1_i)|n1_i-1,n2_i> where |i> = |n1_i, n2_i> and |j> = |n1_j, n2_j>
# b2|n1_i,n2_i> = sqrt(n2_i)|n1_i,n2_i-1> where |i> = |n1_i, n2_i> and |j> = |n1_j, n2_j>
# V = sqrt((n1+1)*(n2+1))*J_n1_n2*(|n1+1,n2><n1,n2+1|+|n1,n2+1><n1+1,n2|)

# states = get_possible_states(3) # up to |3> state
# unsorted_state_energy_dict = { s: w1_b(s[0]) + w2_b(s[1]) for s in states}
# sorted_state_energy_dict = OrderedDict(sorted(unsorted_state_energy_dict.items(), key=lambda x: x[1]))

# sorted_energy = np.array(list(sorted_state_energy_dict.values()))
# sorted_states = list(sorted_state_energy_dict.keys())

sorted_states= [(0,0),(0,1),(1,0),(1,1),(0,2),(2,0),(0,3),(1,2),(2,1),(3,0)] # correct
# sorted_states= [(0,0),(1,0),(0,1),(0,2),(1,1),(2,0),(0,3),(1,2),(2,1),(3,0)]
sorted_energy = [ w1_b(state[0])+w2_b(state[1]) for state in sorted_states]


n = len(sorted_states)
b1_array, b2_array, V0_array = np.zeros((n,n)), np.zeros((n,n)), np.zeros((n,n))

for i, j in itertools.product(range(n), range(n)):
    n1_i, n1_j = sorted_states[i][0], sorted_states[j][0]
    n2_i, n2_j = sorted_states[i][1], sorted_states[j][1]
    
    # b1_array[i,j] = <n1_i, n2_i|b1|n1_j, n2_j>
    b1_array[i, j] = np.sqrt(n1_j)* kronecker_delta(n1_i, n1_j-1) * kronecker_delta(n2_i, n2_j)
    b2_array[i, j] = np.sqrt(n2_j)* kronecker_delta(n2_i, n2_j-1) * kronecker_delta(n1_i, n1_j)
   
    # 1) n1=n1_i-1, n2 = n2_i,  2) n1=n1_i, n2=n2_i-1
    V0_array[i,j] = (np.sqrt(n1_i*(n2_i+1))*J(n1_i-1, n2_i)*kronecker_delta(n1_i-1, n1_j)*kronecker_delta(n2_i+1, n2_j)
                   + np.sqrt((n1_i+1)*n2_i)*J(n1_i, n2_i-1)*kronecker_delta(n1_i+1, n1_j)*kronecker_delta(n2_i-1, n2_j))

H0 = Qobj(np.diag(sorted_energy))    
V0 = Qobj(V0_array)

b1 = Qobj(b1_array)
b2 = Qobj(b2_array)

b1t = b1 + b1.dag()
b2t = b2 + b2.dag()
ntot = b1.dag()*b1 + b2.dag()*b2

In [326]:
sorted_state_energy_dict

OrderedDict([((0, 0), 0.0),
             ((1, 0), 5158.326241134751),
             ((0, 1), 5285.6),
             ((0, 2), 10245.269422917763),
             ((1, 1), 10443.926241134752),
             ((2, 0), 10903.757575757576),
             ((0, 3), 14836.797904403109),
             ((1, 2), 15403.595664052515),
             ((2, 1), 16189.357575757576),
             ((3, 0), 16926.081632653062)])

In [110]:
sorted_energy

array([    0.        ,  5285.6       ,  5158.32624113, 10443.92624113,
       10245.26942292, 10903.75757576, 14836.7979044 , 15403.59566405,
       16189.35757576, 16926.08163265])

In [89]:
sorted_states

Quantum object: dims = [[10], [10]], shape = (10, 10), type = oper, isherm = False
Qobj data =
[[0.         1.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         1.41421356 0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         1.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         1.41421356 0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  1.73205081 0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         1.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0. 

In [312]:
H0
# V0
# b2.dag()

Quantum object: dims = [[10], [10]], shape = (10, 10), type = oper, isherm = True
Qobj data =
[[    0.             0.             0.             0.
      0.             0.             0.             0.
      0.             0.        ]
 [    0.          5285.6            0.             0.
      0.             0.             0.             0.
      0.             0.        ]
 [    0.             0.          5158.32624113     0.
      0.             0.             0.             0.
      0.             0.        ]
 [    0.             0.             0.         10443.92624113
      0.             0.             0.             0.
      0.             0.        ]
 [    0.             0.             0.             0.
  10245.26942292     0.             0.             0.
      0.             0.        ]
 [    0.             0.             0.             0.
      0.         10903.75757576     0.             0.
      0.             0.        ]
 [    0.             0.             0.             0

In [133]:
# states to consider
# 00, 01,10,11,02,20,03,12,21,30
diag = np.array([0, w2_b(1), w1_b(1), w1_b(1)+w2_b(1), w2_b(2), w1_b(2), w2_b(3)
                 ,w1_b(1)+w2_b(2), w1_b(2)+w2_b(1), w1_b(3)])
H0 = Qobj(np.diag(diag))

b1dag = np.zeros((10,10))
b1dag[2,0] = 1
b1dag[3,1] = 1
b1dag[5,2] = np.sqrt(2)
b1dag[8,3] = np.sqrt(2)
b1dag[7,4] = 1
b1dag[9,5] = np.sqrt(3)
b1dag = Qobj(b1dag)

b2dag = np.zeros((10,10))
b2dag[1,0] = 1
b2dag[3,2] = 1
b2dag[4,1] = np.sqrt(2)
b2dag[7,3] = np.sqrt(2)
b2dag[6,4] = np.sqrt(3)
b2dag[8,5] = 1
b2dag = Qobj(b2dag)

V0  = np.zeros((10,10))
V0[2,1] = J(0,0)
V0[1,2] = J(0,0)
V0[4,3] = np.sqrt(2)*J(0,1)
V0[5,3] = np.sqrt(2)*J(1,0)
V0[3,4] = np.sqrt(2)*J(0,1)
V0[3,5] = np.sqrt(2)*J(1,0)
V0[7,6] = np.sqrt(3)*J(0,2)
V0[6,7] = np.sqrt(3)*J(0,2)
V0[8,7] = np.sqrt(4)*J(1,1)
V0[7,8] = np.sqrt(4)*J(1,1)
V0[9,8] = np.sqrt(3)*J(2,0)
V0[8,9] = np.sqrt(3)*J(2,0)
V0 = Qobj(V0)

b1 = b1dag.dag()
b2 = b2dag.dag()

b1t = b1 + b1dag
b2t = b2 + b2dag
ntot = b1dag*b1 + b2dag*b2

In [134]:
V0

Quantum object: dims = [[10], [10]], shape = (10, 10), type = oper, isherm = True
Qobj data =
[[ 0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.        ]
 [ 0.          0.          6.59245214  0.          0.          0.
   0.          0.          0.          0.        ]
 [ 0.          6.59245214  0.          0.          0.          0.
   0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.          8.21399833 13.8625133
   0.          0.          0.          0.        ]
 [ 0.          0.          0.          8.21399833  0.          0.
   0.          0.          0.          0.        ]
 [ 0.          0.          0.         13.8625133   0.          0.
   0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.
   0.          9.1692497   0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.
   9.1692497  

In [135]:
H0

Quantum object: dims = [[10], [10]], shape = (10, 10), type = oper, isherm = True
Qobj data =
[[    0.             0.             0.             0.
      0.             0.             0.             0.
      0.             0.        ]
 [    0.          5285.6            0.             0.
      0.             0.             0.             0.
      0.             0.        ]
 [    0.             0.          5158.32624113     0.
      0.             0.             0.             0.
      0.             0.        ]
 [    0.             0.             0.         10443.92624113
      0.             0.             0.             0.
      0.             0.        ]
 [    0.             0.             0.             0.
  10245.26942292     0.             0.             0.
      0.             0.        ]
 [    0.             0.             0.             0.
      0.         10903.75757576     0.             0.
      0.             0.        ]
 [    0.             0.             0.             0

In [118]:
b2dag

Quantum object: dims = [[10], [10]], shape = (10, 10), type = oper, isherm = False
Qobj data =
[[0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [1.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         1.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         1.41421356 0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         1.73205081 0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         1.41421356 0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0. 

## Diagonalize two-qubit Hamiltonian

In [313]:
# By solving eigenvalue
H1 = H0 + V0
eigen = H1.eigenstates()

# find U
U = sorted_diag_op(eigen)
U_unsort_array = np.column_stack([eigen[1][i] for i in range(eigen[1].size)])
U_unsort = Qobj(U_unsort_array) 

U.dims = H1.dims

# U = U_unsort

# digonalize H0
H2 = U.dag() * H1 * U

[0, 2, 1, 4, 3, 5, 6, 7, 8, 9]


In [304]:
H2
# U
# U_unsort

Quantum object: dims = [[10], [10]], shape = (10, 10), type = oper, isherm = True
Qobj data =
[[    0.             0.             0.             0.
      0.             0.             0.             0.
      0.             0.        ]
 [    0.          5157.98568041     0.             0.
      0.             0.             0.             0.
      0.             0.        ]
 [    0.             0.          5285.94056072     0.
      0.             0.             0.             0.
      0.             0.        ]
 [    0.             0.             0.         10244.92987496
      0.             0.             0.             0.
      0.             0.        ]
 [    0.             0.             0.             0.
  10443.84816286     0.             0.             0.
      0.             0.        ]
 [    0.             0.             0.             0.
      0.         10904.17520199     0.             0.
      0.             0.        ]
 [    0.             0.             0.             0

## Driving Hamiltonian in dressed state

In [321]:
# get new Hd and HR
Omega = 50 # MHz

wd = (H2[3,3]-H2[2,2]+H2[1,1])/2  # 10443 - 5157 + 5285 in MHz(|11>, |10>, |01>)
# wd = (H2[4,4]-H2[1,1]+H2[2,2])/2  # 10443 - 5157 + 5285 in MHz(|11>, |10>, |01>)
Hd = lambda t: Omega * np.cos(2*pi*wd*t)* b1t
Hd_D = lambda t: U.dag() * Hd(t) * U

# To rotating frame
R = lambda t: (-1j*2*pi*wd*ntot*t).expm()
HR = lambda t: R(t).dag() * (H2 + Hd_D(t)) * R(t) - ntot *wd

# average over 10 period
t_list = np.linspace(0, 2, 401)*1e-3  # in us
HR_list = [HR(t) for t in t_list]
H_R = functools.reduce(operator.add, HR_list) / len(HR_list)
H_R.tidyup(1e-4)

Quantum object: dims = [[10], [10]], shape = (10, 10), type = oper, isherm = False
Qobj data =
[[ 0.00000000e+00+0.j          1.29989460e+00-0.00491003j
   2.51628925e+01-0.09504667j  0.00000000e+00+0.j
   0.00000000e+00+0.j          0.00000000e+00+0.j
   0.00000000e+00+0.j          0.00000000e+00+0.j
   0.00000000e+00+0.j          0.00000000e+00+0.j        ]
 [ 1.29989460e+00+0.00491003j  3.90391377e-02+0.j
   0.00000000e+00+0.j          2.50746618e+01-0.0947134j
   1.03769099e+00-0.00391962j -2.59521579e+00+0.00980279j
   0.00000000e+00+0.j          0.00000000e+00+0.j
   0.00000000e+00+0.j          0.00000000e+00+0.j        ]
 [ 2.51628925e+01+0.09504667j  0.00000000e+00+0.j
  -1.27915841e+02+0.j         -2.36941167e+00+0.00894987j
  -8.46146292e-02+0.00031961j -3.55304204e+01+0.13420747j
   0.00000000e+00+0.j          0.00000000e+00+0.j
   0.00000000e+00+0.j          0.00000000e+00+0.j        ]
 [ 0.00000000e+00+0.j          2.50746618e+01+0.0947134j
  -2.36941167e+00-0.00894987j -1

In [315]:
wd

(5285.901521586962+0j)

In [23]:
avg_list = []
for N in range(2,401):
    temp = (functools.reduce(operator.add,[HR_list[i] for i in range(N)]) / N)
    temp = np.real(temp[0,1])
    avg_list.append(temp)
    

In [24]:
 plt.plot(avg_list)
    
# f=np.fft.fft(avg_list)
# plt.plot(np.absolute(f))
# plt.yscale('log')

## Least action

In [322]:
# least action
eigen = H_R.eigenstates()

# find X
X = sorted_diag_op(eigen)
X_unsort_array = np.column_stack([eigen[1][i] for i in range(eigen[1].size)])
X_unsort = Qobj(X_unsort_array) 
# X = X_unsort

# find X_BD and XP
A, B, C = X[0:2,0:2], X[2:4, 2:4], X[4:10, 4:10]
X_BD_array = block_diag(A,B,C)
X_BD = Qobj(X_BD_array)  # X_BD is a block-diagonalization of X. How?
# X_BD = X_BD.tidyup(atol=1e-3)
XP = X_BD * X_BD.dag()

# find T=XF, F=X_BD/sqrt(X_BD*X_BD.dag())
T = X * X_BD.dag() * XP.sqrtm().inv()

# find H_R_BD
T.dims = H_R.dims
H_R_BD =  T.dag() * H_R * T
H_R_BD.tidyup(1e-4)

[6, 7, 4, 3, 2, 1, 0, 8, 5, 9]


Quantum object: dims = [[10], [10]], shape = (10, 10), type = oper, isherm = True
Qobj data =
[[ 4.64430544e+00+0.j          1.12777848e+00-0.00425991j
   0.00000000e+00+0.j          0.00000000e+00+0.j
   0.00000000e+00+0.j          0.00000000e+00+0.j
   0.00000000e+00+0.j          0.00000000e+00+0.j
   0.00000000e+00+0.j          0.00000000e+00+0.j        ]
 [ 1.12777848e+00+0.00425991j  4.63211583e+00+0.j
   0.00000000e+00+0.j          0.00000000e+00+0.j
   0.00000000e+00+0.j          0.00000000e+00+0.j
   0.00000000e+00+0.j          0.00000000e+00+0.j
   0.00000000e+00+0.j          0.00000000e+00+0.j        ]
 [ 0.00000000e+00+0.j          0.00000000e+00+0.j
  -1.35308699e+02+0.j         -2.20939031e+00+0.00834543j
   0.00000000e+00+0.j          0.00000000e+00+0.j
   0.00000000e+00+0.j          0.00000000e+00+0.j
   0.00000000e+00+0.j          0.00000000e+00+0.j        ]
 [ 0.00000000e+00+0.j          0.00000000e+00+0.j
  -2.20939031e+00-0.00834543j -1.35345488e+02+0.j
   0.00000000

In [324]:
X.tidyup(1e-3)
# X_unsort.tidyup(1e-4)

Quantum object: dims = [[10], [10]], shape = (10, 10), type = oper, isherm = False
Qobj data =
[[ 0.69776416+0.j         -0.69265965+0.00261635j -0.13273671+0.j
  -0.12524518+0.j          0.        +0.j         -0.00505923+0.j
   0.        +0.j          0.        +0.j          0.002968  +0.j
   0.        +0.j        ]
 [ 0.69406632+0.00262167j  0.69648618+0.j          0.13220916+0.j
  -0.12483709+0.j         -0.00323384+0.j         -0.00385647+0.j
   0.        +0.j          0.        +0.j          0.00909786+0.j
   0.        +0.j        ]
 [ 0.12403372+0.j         -0.132611  +0.j          0.69537857+0.j
   0.69102999-0.00261019j  0.        +0.j         -0.06697025+0.j
   0.        +0.j          0.        +0.j          0.03822017+0.j
   0.00172507+0.j        ]
 [ 0.12424636+0.j          0.13132881+0.j         -0.68965584-0.002605j
   0.69685992+0.j          0.        +0.j          0.03835076+0.j
   0.        +0.j          0.        +0.j          0.06778465+0.j
   0.00140896+0.j        ]

## Pauli coefficient

In [295]:
def from16to8(M):
    """
    Convert 4x4 block-diagonal matrix to 8x1 column vector
    Args:
        M: 4x4 block-diagonal matrix
    Return:
        8x1 matrix
    """
    return np.array([M[0,0],M[1,0],M[0,1],M[1,1],M[2,2],M[3,2],M[2,3],M[3,3]]).reshape(8,1)

# Get Pauli coeff.
# All these are block-diagonal.
II = tensor(qeye(2), qeye(2))
IX = tensor(qeye(2), sigmax())
IY = tensor(qeye(2), sigmay())
IZ = tensor(qeye(2), sigmaz())
ZI = tensor(sigmaz(), qeye(2))
ZY = tensor(sigmaz(), sigmay())
ZX = tensor(sigmaz(), sigmax())
ZZ = tensor(sigmaz(), sigmaz())

a = np.column_stack([from16to8(II),from16to8(IX)/2,from16to8(IY)/2,from16to8(IZ)/2
                    ,from16to8(ZI)/2,from16to8(ZX)/2,from16to8(ZY)/2,from16to8(ZZ)/2])

b = from16to8(H_R_BD[0:4, 0:4])

c = np.linalg.solve(a, b)
sympy.Matrix(np.round(c,6))
# -63.9, -0.02, -0.00017,1.2e-5,127.9,0.073, 0.00017, -0.039

Matrix([
[-64.061899],
[  0.524024],
[  0.001918],
[  0.001011],
[  127.9531],
[  0.482508],
[  0.001918],
[  0.169686]])

## Pauli vs CR amplitude

In [296]:
def get_Pauli_coeff(Omega):
    # get new Hd and HR
#     Omega = 1 # MHz
    wd = (H2[3,3]-H2[2,2]+H2[1,1])/2  # 10443 - 5157 + 5285 in MHz
    Hd = lambda t: Omega * np.cos(2*pi*wd*t)* b1t
    Hd_D = lambda t: U.dag() * Hd(t) * U

    # To rotating frame
    R = lambda t: (-1j*2*pi*wd*ntot*t).expm()
    HR = lambda t: R(t).dag() * (H2 + Hd_D(t)) * R(t) - ntot *wd

    # average over 10 period
    t_list = np.linspace(0, 2, 401)*1e-3  # in us
    HR_list = [HR(t) for t in t_list]
    H_R = functools.reduce(operator.add, HR_list) / len(HR_list)
    
    # least action
    eigen = H_R.eigenstates()

    # find X
    X = sorted_diag_op(eigen)

    # find X_BD and XP
    A, B, C = X[0:2,0:2], X[2:4, 2:4], X[4:10, 4:10]
    X_BD_array = block_diag(A,B,C)
    X_BD = Qobj(X_BD_array)
    # X_BD = X_BD.tidyup(atol=1e-3)
    XP = X_BD * X_BD.conj().trans()

    # find T
    T = X * X_BD.dag() * XP.sqrtm().inv()

    # find H_R_BD
    T.dims = H_R.dims
    H_R_BD =  T.dag() * H_R * T
   
    def from16to8(M):
        return np.array([M[0,0],M[1,0],M[0,1],M[1,1],M[2,2],M[3,2],M[2,3],M[3,3]]).reshape(8,1)

    # Get Pauli coeff.
    II = tensor(qeye(2), qeye(2))
    IX = tensor(qeye(2), sigmax())
    IY = tensor(qeye(2), sigmay())
    IZ = tensor(qeye(2), sigmaz())
    ZI = tensor(sigmaz(), qeye(2))
    ZY = tensor(sigmaz(), sigmay())
    ZX = tensor(sigmaz(), sigmax())
    ZZ = tensor(sigmaz(), sigmaz())

    a = np.column_stack([from16to8(II),from16to8(IX)/2,from16to8(IY)/2,from16to8(IZ)/2
                        ,from16to8(ZI)/2,from16to8(ZX)/2,from16to8(ZY)/2,from16to8(ZZ)/2])

    b = from16to8(H_R_BD[0:4, 0:4])

    c = np.linalg.solve(a, b)
    print(Omega)
    print(X_BD[0,0], X_BD[1,1], X_BD[2,2], X_BD[3,3])
    return c

In [297]:
# plot Pauli coeff. vs CR amplitude
Omega_list = np.linspace(1, 140, 11)
Pauli = get_Pauli_coeff(Omega_list[0])
for omega in Omega_list[1:]:
    Pauli = np.column_stack((Pauli, get_Pauli_coeff(omega)))

# plot
Pauli_label = ['II','IX','IY','IZ','ZI','ZX','ZY','ZZ']
fig, ax = plt.subplots(1,1, figsize=(10,8))
for i in range(Pauli.shape[0]):
    ax.plot(Omega_list, Pauli[i], label=Pauli_label[i], linewidth=3)
ax.set_xlabel('CR amplitude, Omega (MHz)', fontsize=18)
ax.set_ylabel('$Pauli Coefficient$ (MHz)', fontsize=18)
ax.tick_params(axis='x', labelsize=18)
ax.tick_params(axis='y', labelsize=18)
ax.grid('on')
ax.legend(fontsize=18)

[6, 7, 4, 2, 3, 1, 0, 8, 5, 9]
1.0
(0.7639392475715586+0j) (0.7639375098408965+0j) (0.9927255837553963+0j) (0.9927275926005514+0j)
[6, 7, 4, 2, 3, 1, 0, 8, 5, 9]
14.9
(0.7190025297849294+0j) (0.7186543962861673+0j) (0.739986913025513+0j) (0.7403222225377037+0j)
[6, 7, 4, 3, 2, 1, 0, 8, 5, 9]
28.8
(0.7243992876228106+0j) (0.7231453418963387+0j) (0.7705905166336141+0j) (0.7718133899716885+0j)
[6, 7, 4, 3, 2, 1, 0, 8, 5, 9]
42.7
(0.730896807614905+0j) (0.7282399521260473+0j) (0.8241917517341232+0j) (0.8269592011935631+0j)
[6, 7, 4, 3, 2, 1, 0, 8, 5, 9]
56.6
(0.7375455004572835+0j) (0.7330515144728841+0j) (0.8585151995702691+0j) (0.8633462767662169+0j)
[6, 7, 4, 3, 2, 1, 0, 8, 5, 9]
70.5
(0.7441175815831084+0j) (0.7374118766766734+0j) (0.8808460255673809+0j) (0.8881220121814287+0j)
[6, 7, 4, 3, 2, 1, 0, 8, 5, 9]
84.4
(0.7505127681943283+0j) (0.7412697739624732+0j) (0.8949775822439403+0j) (0.9049422527828237+0j)
[6, 7, 4, 3, 2, 1, 0, 8, 5, 9]
98.3
(0.7566686879498252+0j) (0.744583236788864+

ValueError: 5 is not in list

In [55]:
# plot ZZ only

fig, ax = plt.subplots(1,1, figsize=(10,8))
i = 7
ax.plot(Omega_list, Pauli[i], label=Pauli_label[i], linewidth=3)
ax.set_xlabel('CR amplitude, Omega (MHz)', fontsize=18)
ax.set_ylabel(r'$\zeta$ (MHz)', fontsize=18)
ax.tick_params(axis='x', labelsize=18)
ax.tick_params(axis='y', labelsize=18)
ax.grid('on')
ax.legend(fontsize=18)

## Using Duffing model

In [ ]:
wm = 5164       # CSFQ w01
del_m0 = 600   # first anharmonicity
del_m1 = 430  # second anharmon.
gdm = 80     # CSFQ-bus coupling

wr = 6292 # bus cavity

wt = 5292   # transmon
del_t0 = -329.1
del_t1 = -369.7
gdt = -80    # transmon-bus coupling

gmt = 0  # csfq-transmon direct g

# CSFQ, j=0,1,2
w1_d = lambda j: wm*j + del_m0/2*j*(j-1) if j<3 else wm*j + del_m0/2*j*(j-1) + del_m1-del_m0 # dressed freq
gamma1 = lambda j: j/(wr + w1_d(j-1) - w1_d(j))
w1_b = lambda j: w1_d(j) - gamma1(j) * gdm**2  # bare freq. j=1,2,3

# transmon, j=0,1,2
w2_d = lambda j: wt*j + del_t0/2*j*(j-1) if j<3 else wt*j + del_t0/2*j*(j-1) + del_t1-del_t0 # dressed freq
gamma2 = lambda j: j/(wr + w2_d(j-1) - w2_d(j))
w2_b = lambda j: w2_d(j) - gamma2(j) * gdt**2  # bare freq.

# J, {j1,j2}=0,1,2
J = lambda j1,j2 : gmt - gdm*gdt/2*(1/(wr+w1_d(j1)-w1_d(j1+1)) + 1/(wr+w2_d(j2)-w2_d(j2+1)) 
                                    + 1/(wr-w1_d(j1)+w1_d(j1+1)) + 1/(wr-w2_d(j2)+w2_d(j2+1))) 

##############
def get_possible_states(n):
    """
    Get a list of tuples of possible states, given n
    Ex) for n=2, output is [(0,0),(0,1),(1,0),(0,2),(1,1),(2,0)]
    Args: 
        n: integer such that n1+n2<=n for |n1,n2> state
    Return:
        List of tuples that satisfy n1+n2<=n where n1 and n2 are 0,1,2,3,....
    """
    def get_possible_sum(n):
        """
         [(0,1)]
        """
        result = []
        for i in range(n+1):
            result.append((i,n-i))
        return result
    
    possible_list = []
    for i in range(n + 1):
        possible_list += get_possible_sum(i)
    
    return possible_list
def kronecker_delta(i,j):
    return (1 if i==j else 0)

# Note
# b1[i,j] = <i|b1|j>, where |i> = {|00>, |01>, |10>, |11>, ... |30>}
# b1|n1_i,n2_i> = sqrt(n1_i)|n1_i-1,n2_i> where |i> = |n1_i, n2_i> and |j> = |n1_j, n2_j>
# b2|n1_i,n2_i> = sqrt(n2_i)|n1_i,n2_i-1> where |i> = |n1_i, n2_i> and |j> = |n1_j, n2_j>
# V = sqrt((n1+1)*(n2+1))*J_n1_n2*(|n1+1,n2><n1,n2+1|+|n1,n2+1><n1+1,n2|)

states = get_possible_states(3) # up to |3> state
unsorted_state_energy_dict = { s: w1_b(s[0]) + w2_b(s[1]) for s in states}
sorted_state_energy_dict = OrderedDict(sorted(unsorted_state_energy_dict.items(), key=lambda x: x[1]))

sorted_energy = np.array(list(sorted_state_energy_dict.values()))
sorted_states = list(sorted_state_energy_dict.keys())

# sorted_states= [(0,0),(0,1),(1,0),(1,1),(0,2),(2,0),(0,3),(1,2),(2,1),(3,0)]
# sorted_energy = np.array([0, w2_b(1), w1_b(1), w1_b(1)+w2_b(1), w2_b(2), w1_b(2), w2_b(3)
#                  ,w1_b(1)+w2_b(2), w1_b(2)+w2_b(1), w1_b(3)])

n = len(sorted_state_energy_dict)
b1_array, b2_array, V0_array = np.zeros((n,n)), np.zeros((n,n)), np.zeros((n,n))

for i, j in itertools.product(range(n), range(n)):
    n1_i, n1_j = sorted_states[i][0], sorted_states[j][0]
    n2_i, n2_j = sorted_states[i][1], sorted_states[j][1]
    
    # b1_array[i,j] = <n1_i, n2_i|b1|n1_j, n2_j>
    b1_array[i, j] = np.sqrt(n1_j)* kronecker_delta(n1_i, n1_j-1) * kronecker_delta(n2_i, n2_j)
    b2_array[i, j] = np.sqrt(n2_j)* kronecker_delta(n2_i, n2_j-1) * kronecker_delta(n1_i, n1_j)
   
    # 1) n1=n1_i-1, n2 = n2_i,  2) n1=n1_i, n2=n2_i-1
    V0_array[i,j] = (np.sqrt(n1_i*(n2_i+1))*J(n1_i-1, n2_i)*kronecker_delta(n1_i-1, n1_j)*kronecker_delta(n2_i+1, n2_j)
                   + np.sqrt((n1_i+1)*n2_i)*J(n1_i, n2_i-1)*kronecker_delta(n1_i+1, n1_j)*kronecker_delta(n2_i-1, n2_j))

def hamiltonian_eff_H0(w1,w2,J,alpha1,alpha2,N):
    """ 
    Two-transmon effective Hamiltonian with zero excitation in bus. 
    Eq. 2.12, Magesan
    w1, w2:  angular freq
    """
    b1 = destroy(N)
    b2 = destroy(N)
    
    I = qeye(N)

    # Eq. 2.21, Magesan
    H1 = w1*tensor(b1.dag()*b1, I) + alpha1/2.0*tensor(b1.dag()*b1.dag()*b1*b1, I)
    H2 = w2*tensor(I, b2.dag()*b2) + alpha2/2.0*tensor(I, b2.dag()*b2.dag()*b2*b2)
    H12 = J*(tensor(b1.dag(), b2) + tensor(b1, b2.dag())) 
    
    H_eff = H1 + H2 + H12
    
    return H_eff

H0 = Qobj(np.diag(sorted_energy))    
V0 = Qobj(V0_array)

b1 = Qobj(b1_array)
b2 = Qobj(b2_array)

b1t = b1 + b1.dag()
b2t = b2 + b2.dag()
ntot = b1.dag()*b1 + b2.dag()*b2
###############

# By solving eigenvalue
H1 = H0 + V0
eigen = H1.eigenstates()

# find U
U = sorted_diag_op(eigen)
U_unsort_array = np.column_stack([eigen[1][i] for i in range(eigen[1].size)])
U_unsort = Qobj(U_unsort_array) 
U.dims = H1.dims

# digonalize H0
H2 = U.dag() * H1 * U
#########

# get new Hd and HR
Omega = 1 # MHz
wd = (H2[3,3]-H2[2,2]+H2[1,1])/2  # 10443 - 5157 + 5285 in MHz
Hd = lambda t: Omega * np.cos(2*pi*wd*t)* b1t
Hd_D = lambda t: U.dag() * Hd(t) * U

# To rotating frame
R = lambda t: (-1j*2*pi*wd*ntot*t).expm()
HR = lambda t: R(t).dag() * (H2 + Hd_D(t)) * R(t) - ntot *wd

# average over 10 period
t_list = np.linspace(0, 2, 401)*1e-3  # in us
HR_list = [HR(t) for t in t_list]
H_R = functools.reduce(operator.add, HR_list) / len(HR_list)
H_R
##############

# least action
eigen = H_R.eigenstates()

# find X
X = sorted_diag_op(eigen)

# find X_BD and XP
A, B, C = X[0:2,0:2], X[2:4, 2:4], X[4:10, 4:10]
X_BD_array = block_diag(A,B,C)
X_BD = Qobj(X_BD_array)
# X_BD = X_BD.tidyup(atol=1e-3)
XP = X_BD * X_BD.conj().trans()

# find T
T = X * X_BD.dag() * XP.sqrtm().inv()

# find H_R_BD
T.dims = H_R.dims
H_R_BD =  T.dag() * H_R * T
H_R_BD.tidyup(1e-4)
##################

def from16to8(M):
    """
    Convert 4x4 block-diagonal matrix to 8x1 column vector
    Args:
        M: 4x4 block-diagonal matrix
    Return:
        8x1 matrix
    """
    return np.array([M[0,0],M[1,0],M[0,1],M[1,1],M[2,2],M[3,2],M[2,3],M[3,3]]).reshape(8,1)

# Get Pauli coeff.
# All these are block-diagonal.
II = tensor(qeye(2), qeye(2))
IX = tensor(qeye(2), sigmax())
IY = tensor(qeye(2), sigmay())
IZ = tensor(qeye(2), sigmaz())
ZI = tensor(sigmaz(), qeye(2))
ZY = tensor(sigmaz(), sigmay())
ZX = tensor(sigmaz(), sigmax())
ZZ = tensor(sigmaz(), sigmaz())

a = np.column_stack([from16to8(II),from16to8(IX)/2,from16to8(IY)/2,from16to8(IZ)/2
                    ,from16to8(ZI)/2,from16to8(ZX)/2,from16to8(ZY)/2,from16to8(ZZ)/2])

b = from16to8(H_R_BD[0:4, 0:4])

c = np.linalg.solve(a, b)
sympy.Matrix(np.round(c,6))
# -63.9, -0.02, -0.00017,1.2e-5,127.9,0.073, 0.00017, -0.039


# Function : get_Pauli_coefficient

In [116]:
def sorted_diag_op(eigen):
    """
    Diagonalizing matrix by the order of eigenvector
    
    Args:
        eigen : output of eigenstates() method in qutip
       
    Return: a new diagonalizing operator Qobj
    """
       
    index_list = [np.argmax(np.absolute(vec)) for vec in eigen[1]]
    
    X_array = np.column_stack([eigen[1][index_list.index(i)] for i in range(eigen[1].size)])
    X = Qobj(X_array)
    
    return X

def get_Pauli_coefficient(wm,del_m0,del_m1,gdm
                         ,wt,del_t0,del_t1,gdt
                         ,wr,gmt,Omega):

    # CSFQ, j=0,1,2
    w1_d = lambda j: wm*j + del_m0/2*j*(j-1) if j<3 else wm*j + del_m0/2*j*(j-1) + del_m1-del_m0 # dressed freq
    gamma1 = lambda j: j/(wr + w1_d(j-1) - w1_d(j))
    w1_b = lambda j: w1_d(j) - gamma1(j) * gdm**2  # bare freq. j=1,2,3

    # transmon, j=0,1,2
    w2_d = lambda j: wt*j + del_t0/2*j*(j-1) if j<3 else wt*j + del_t0/2*j*(j-1) + del_t1-del_t0 # dressed freq
    gamma2 = lambda j: j/(wr + w2_d(j-1) - w2_d(j))
    w2_b = lambda j: w2_d(j) - gamma2(j) * gdt**2  # bare freq.

    # J, {j1,j2}=0,1,2
    J = lambda j1,j2 : gmt - gdm*gdt/2*(1/(wr+w1_d(j1)-w1_d(j1+1)) + 1/(wr+w2_d(j2)-w2_d(j2+1)) 
                                        + 1/(wr-w1_d(j1)+w1_d(j1+1)) + 1/(wr-w2_d(j2)+w2_d(j2+1))) 
    
    def get_possible_states(n):
        """
        Get a list of tuples of possible states, given n
        Ex) for n=2, output is [(0,0),(0,1),(1,0),(0,2),(1,1),(2,0)]
        Args: 
            n: integer such that n1+n2<=n for |n1,n2> state
        Return:
            List of tuples that satisfy n1+n2<=n where n1 and n2 are 0,1,2,3,....
        """
        def get_possible_sum(n):
                """
                 [(0,1)]
                """
                result = []
                for i in range(n+1):
                    result.append((i,n-i))
                return result

        possible_list = []
        for i in range(n + 1):
            possible_list += get_possible_sum(i)

        return possible_list
    def kronecker_delta(i,j):
        return (1 if i==j else 0)

    # Get H0, V0, b1 and b2
    states = get_possible_states(3) # up to |3> state
    unsorted_state_energy_dict = { s: w1_b(s[0]) + w2_b(s[1]) for s in states}
   
    sorted_state_energy_dict = OrderedDict(sorted(unsorted_state_energy_dict.items(), key=lambda x: x[1]))
    sorted_energy = np.array(list(sorted_state_energy_dict.values()))
    sorted_states = list(sorted_state_energy_dict.keys())

#     Xuexin
    sorted_states= [(0,0),(0,1),(1,0),(1,1),(0,2),(2,0),(0,3),(1,2),(2,1),(3,0)]
    sorted_energy = np.array([0, w2_b(1), w1_b(1), w1_b(1)+w2_b(1), w2_b(2), w1_b(2), w2_b(3)
                 ,w1_b(1)+w2_b(2), w1_b(2)+w2_b(1), w1_b(3)])

    n = len(sorted_state_energy_dict)
    b1_array, b2_array, V0_array = np.zeros((n,n)), np.zeros((n,n)), np.zeros((n,n))

    for i, j in itertools.product(range(n), range(n)):
        n1_i, n1_j = sorted_states[i][0], sorted_states[j][0]
        n2_i, n2_j = sorted_states[i][1], sorted_states[j][1]

        # b1_array[i,j] = <n1_i, n2_i|b1|n1_j, n2_j>
        b1_array[i, j] = np.sqrt(n1_j)* kronecker_delta(n1_i, n1_j-1) * kronecker_delta(n2_i, n2_j)
        b2_array[i, j] = np.sqrt(n2_j)* kronecker_delta(n2_i, n2_j-1) * kronecker_delta(n1_i, n1_j)

        # 1) n1=n1_i-1, n2 = n2_i,  2) n1=n1_i, n2=n2_i-1
        V0_array[i,j] = (np.sqrt(n1_i*(n2_i+1))*J(n1_i-1, n2_i)*kronecker_delta(n1_i-1, n1_j)*kronecker_delta(n2_i+1, n2_j)
                       + np.sqrt((n1_i+1)*n2_i)*J(n1_i, n2_i-1)*kronecker_delta(n1_i+1, n1_j)*kronecker_delta(n2_i-1, n2_j))

    H0 = Qobj(np.diag(sorted_energy))    
    V0 = Qobj(V0_array)

    b1 = Qobj(b1_array)
    b2 = Qobj(b2_array)
    
    b1t = b1 + b1.dag()
    b2t = b2 + b2.dag()
    ntot = b1.dag()*b1 + b2.dag()*b2

    # By solving eigenvalue
    H1 = H0 + V0
    eigen = H1.eigenstates()

    # find U
    U = sorted_diag_op(eigen)
    U.dims = H1.dims

    # digonalize H0
    H2 = U.dag() * H1 * U
    
    # get new Hd and HR
#     Omega = 1 # MHz
    wd = (H2[3,3]-H2[2,2]+H2[1,1])/2  # 10443 - 5157 + 5285 in MHz
    Hd = lambda t: Omega * np.cos(2*pi*wd*t)* b1t
    Hd_D = lambda t: U.dag() * Hd(t) * U

    # To rotating frame
    R = lambda t: (-1j*2*pi*wd*ntot*t).expm()
    HR = lambda t: R(t).dag() * (H2 + Hd_D(t)) * R(t) - ntot *wd

    # average over 10 period
    t_list = np.linspace(0, 2, 401)*1e-3  # in us
    HR_list = [HR(t) for t in t_list]
    H_R = functools.reduce(operator.add, HR_list) / len(HR_list)
        
    # least action
    eigen = H_R.eigenstates()

    # find X
    X = sorted_diag_op(eigen)

    # find X_BD and XP
    A, B, C = X[0:2,0:2], X[2:4, 2:4], X[4:10, 4:10]
    X_BD_array = block_diag(A,B,C)
    X_BD = Qobj(X_BD_array)
    # X_BD = X_BD.tidyup(atol=1e-3)
    XP = X_BD * X_BD.dag()

    # find T
    T = X * X_BD.dag() * XP.sqrtm().inv()

    # find H_R_BD
    T.dims = H_R.dims
    H_R_BD =  T.dag() * H_R * T
       
    def from16to8(M):
        return np.array([M[0,0],M[1,0],M[0,1],M[1,1],M[2,2],M[3,2],M[2,3],M[3,3]]).reshape(8,1)

    # Get Pauli coeff.
    II = tensor(qeye(2), qeye(2))
    IX = tensor(qeye(2), sigmax())
    IY = tensor(qeye(2), sigmay())
    IZ = tensor(qeye(2), sigmaz())
    ZI = tensor(sigmaz(), qeye(2))
    ZY = tensor(sigmaz(), sigmay())
    ZX = tensor(sigmaz(), sigmax())
    ZZ = tensor(sigmaz(), sigmaz())

    a = np.column_stack([from16to8(II),from16to8(IX)/2,from16to8(IY)/2,from16to8(IZ)/2
                        ,from16to8(ZI)/2,from16to8(ZX)/2,from16to8(ZY)/2,from16to8(ZZ)/2])

    b = from16to8(H_R_BD[0:4, 0:4])

    c = np.linalg.solve(a, b)
    
    return c

In [119]:
# parameters CSFQ-transmon
wm = 5164       # CSFQ w01
del_m0 = 600   # first anharmonicity
del_m1 = 430  # second anharmon.
gdm = 80     # CSFQ-bus coupling

wr = 6292 # bus cavity

wt = 5292   # transmon
del_t0 = -329.1
del_t1 = -369.7
gdt = 80    # transmon-bus coupling

gmt = 0  # csfq-transmon direct g

In [124]:
# parameters, transmon-transmon
wm = 5292 #5164       # CSFQ w01
del_m0 = -330 #600   # first anharmonicity
del_m1 = -370 #430  # second anharmon.
gdm = 80     # CSFQ-bus coupling

wr = 6292 # bus cavity

wt = 5164 #5292   # transmon
del_t0 = -329.1
del_t1 = -369.7
gdt = 80    # transmon-bus coupling

gmt = 0  # csfq-transmon direct g

In [54]:
# parameters, transmon-transmon (usual target-control swapped)
wm = 5164       # CSFQ w01
del_m0 = -330 #600   # first anharmonicity
del_m1 = -370 #430  # second anharmon.
gdm = 80    # CSFQ-bus coupling

wr = 6292 # bus cavity

wt = 5292   # transmon
del_t0 = -329.1
del_t1 = -369.7
gdt = -80   # transmon-bus coupling

gmt = 2.7  # csfq-transmon direct g

In [125]:
# plot Pauli coeff. vs CR amplitude
Omega_list = np.linspace(0, 200, 20)

Pauli = get_Pauli_coefficient(wm,del_m0,del_m1,gdm
                         ,wt,del_t0,del_t1,gdt
                         ,wr,gmt,Omega_list[0])
for omega in Omega_list[1:]:
    Pauli = np.column_stack((Pauli, get_Pauli_coefficient(wm,del_m0,del_m1,gdm
                         ,wt,del_t0,del_t1,gdt,wr,gmt,omega)))
    
# plot
Pauli_label = ['II','IX','IY','IZ','ZI','ZX','ZY','ZZ']
fig, ax = plt.subplots(1,1, figsize=(10,8))
for i in range(Pauli.shape[0]):
    ax.plot(Omega_list, Pauli[i], label=Pauli_label[i], linewidth=3)
ax.set_xlabel('CR amplitude, Omega (MHz)', fontsize=18)
ax.set_ylabel('$Pauli Coefficient$ (MHz)', fontsize=18)
ax.tick_params(axis='x', labelsize=18)
ax.tick_params(axis='y', labelsize=18)
ax.grid('on')
ax.legend(fontsize=18)

/usr/lib/python3/dist-packages/numpy/core/_asarray.py:85: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)


# Function : get ZZ

In [6]:
def get_static_ZZ(wm,del_m0,del_m1,gdm
                  ,wt,del_t0,del_t1,gdt
                         ,wr,gmt):
    """
    By diagonalizing two-qubit Hamiltonian, calculate ZZ(=E11-E01-E10+E00).
    """
    
    # CSFQ, j=0,1,2
    w1_d = lambda j: wm*j + del_m0/2*j*(j-1) if j<3 else wm*j + del_m0/2*j*(j-1) + del_m1-del_m0 # dressed freq
    gamma1 = lambda j: j/(wr + w1_d(j-1) - w1_d(j))
    w1_b = lambda j: w1_d(j) - gamma1(j) * gdm**2  # bare freq. j=1,2,3

    # transmon, j=0,1,2
    w2_d = lambda j: wt*j + del_t0/2*j*(j-1) if j<3 else wt*j + del_t0/2*j*(j-1) + del_t1-del_t0 # dressed freq
    gamma2 = lambda j: j/(wr + w2_d(j-1) - w2_d(j))
    w2_b = lambda j: w2_d(j) - gamma2(j) * gdt**2  # bare freq.

    # J, {j1,j2}=0,1,2
    J = lambda j1,j2 : gmt - gdm*gdt/2*(1/(wr+w1_d(j1)-w1_d(j1+1)) + 1/(wr+w2_d(j2)-w2_d(j2+1)) 
                                        + 1/(wr-w1_d(j1)+w1_d(j1+1)) + 1/(wr-w2_d(j2)+w2_d(j2+1))) 
    
    def get_possible_states(n):
        """
        Get a list of tuples of possible states, given n
        Ex) for n=2, output is [(0,0),(0,1),(1,0),(0,2),(1,1),(2,0)]
        Args: 
            n: integer such that n1+n2<=n for |n1,n2> state
        Return:
            List of tuples that satisfy n1+n2<=n where n1 and n2 are 0,1,2,3,....
        """
        def get_possible_sum(n):
                """
                 [(0,1)]
                """
                result = []
                for i in range(n+1):
                    result.append((i,n-i))
                return result

        possible_list = []
        for i in range(n + 1):
            possible_list += get_possible_sum(i)

        return possible_list
    def kronecker_delta(i,j):
        return (1 if i==j else 0)   
    def sorted_diag_op(eigen):
        """
        Diagonalizing matrix by the order of eigenvector

        Args:
            eigen : output of eigenstates() method in qutip

        Return: a new diagonalizing operator Qobj
        """

        index_list = [np.argmax(np.absolute(vec)) for vec in eigen[1]]

        X_array = np.column_stack([eigen[1][index_list.index(i)] for i in range(eigen[1].size)])
        X = Qobj(X_array)

        return X
    
    # Get H0, V0, b1 and b2
    states = get_possible_states(3) # up to |3> state
    unsorted_state_energy_dict = { s: w1_b(s[0]) + w2_b(s[1]) for s in states}
   
    sorted_state_energy_dict = OrderedDict(sorted(unsorted_state_energy_dict.items(), key=lambda x: x[1]))
    sorted_energy = np.array(list(sorted_state_energy_dict.values()))
    sorted_states = list(sorted_state_energy_dict.keys())

#     Xuexin
    sorted_states= [(0,0),(0,1),(1,0),(1,1),(0,2),(2,0),(0,3),(1,2),(2,1),(3,0)]
    sorted_energy = np.array([0, w2_b(1), w1_b(1), w1_b(1)+w2_b(1), w2_b(2), w1_b(2), w2_b(3)
                 ,w1_b(1)+w2_b(2), w1_b(2)+w2_b(1), w1_b(3)])

    n = len(sorted_state_energy_dict)
    b1_array, b2_array, V0_array = np.zeros((n,n)), np.zeros((n,n)), np.zeros((n,n))

    for i, j in itertools.product(range(n), range(n)):
        n1_i, n1_j = sorted_states[i][0], sorted_states[j][0]
        n2_i, n2_j = sorted_states[i][1], sorted_states[j][1]

        # b1_array[i,j] = <n1_i, n2_i|b1|n1_j, n2_j>
        b1_array[i, j] = np.sqrt(n1_j)* kronecker_delta(n1_i, n1_j-1) * kronecker_delta(n2_i, n2_j)
        b2_array[i, j] = np.sqrt(n2_j)* kronecker_delta(n2_i, n2_j-1) * kronecker_delta(n1_i, n1_j)

        # 1) n1=n1_i-1, n2 = n2_i,  2) n1=n1_i, n2=n2_i-1
        V0_array[i,j] = (np.sqrt(n1_i*(n2_i+1))*J(n1_i-1, n2_i)*kronecker_delta(n1_i-1, n1_j)*kronecker_delta(n2_i+1, n2_j)
                       + np.sqrt((n1_i+1)*n2_i)*J(n1_i, n2_i-1)*kronecker_delta(n1_i+1, n1_j)*kronecker_delta(n2_i-1, n2_j))

    H0 = Qobj(np.diag(sorted_energy))    
    V0 = Qobj(V0_array)

    b1 = Qobj(b1_array)
    b2 = Qobj(b2_array)
    
    b1t = b1 + b1.dag()
    b2t = b2 + b2.dag()
    ntot = b1.dag()*b1 + b2.dag()*b2

    # By solving eigenvalue
    H1 = H0 + V0
    eigen = H1.eigenstates()

    # find U
    U = sorted_diag_op(eigen)
    U.dims = H1.dims

    # digonalize H0
    H2 = U.dag() * H1 * U
    
    return (H2[3,3]-H2[2,2]-H2[1,1], J(0,0))

In [17]:
# parameters, transmon-transmon
wm = 5292 #5164       # transmon
del_m0 = -330 #600   # first anharmonicity
del_m1 = -370 #430  # second anharmon.
gdm = 120     # transmon-bus coupling

wr = 6292 # bus cavity

wt = 5164 #5292   # transmon
del_t0 = -329.1
del_t1 = -369.7
gdt = 120    # transmon-bus coupling

gmt = 3  # csfq-transmon direct g

In [18]:
get_static_ZZ(wm,del_m0,del_m1,gdm
              ,wt,del_t0,del_t1,gdt
              ,wr,gmt)[1]

-11.833017304841954

In [19]:
# plot Pauli coeff. vs CR amplitude
wr_list = np.linspace(5500, 8000, 101)

ZZ = [get_static_ZZ(wm,del_m0,del_m1,gdm
                   ,wt,del_t0,del_t1,gdt
                   ,wr,gmt)[0] for wr in wr_list]
J = [get_static_ZZ(wm,del_m0,del_m1,gdm
                   ,wt,del_t0,del_t1,gdt
                   ,wr,gmt)[1] for wr in wr_list]  
# plot
fig, ax = plt.subplots(1,1, figsize=(10,8))
ax.plot(wr_list, ZZ, linewidth=3, label='ZZ')
ax.plot(wr_list, J, linewidth=3, label='J')
ax.set_xlabel('wr (MHz)', fontsize=18)
ax.set_ylabel('$ZZ$ (MHz)', fontsize=18)
ax.tick_params(axis='x', labelsize=18)
ax.tick_params(axis='y', labelsize=18)
ax.grid('on')
ax.legend(fontsize=18)

/usr/lib/python3/dist-packages/numpy/core/_asarray.py:85: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)


In [100]:
plt.plot(wr_list, J)

# Test

In [132]:
from sympy import I
Omega = 10
wd = sympy.Symbol('wd')
t = sympy.Symbol('t')

Hd = Omega * (sympy.exp(-I*wd*t) + sympy.exp(I*wd*t))/2 * sympy.Matrix(b1t.data.toarray())
Hdt = (U.dag()).data.toarray() * Hd * U.data.toarray()

Htot = H2.data.toarray() + Hdt

Ur = sympy.exp(-I*wd *sympy.Matrix(ntot.data.toarray())*t)
Urdag = sympy.exp(I*wd *sympy.Matrix(ntot.data.toarray())*t)
H3t = Urdag * Htot * Ur -  sympy.Matrix(ntot.data.toarray())

H3t_RWA = sympy.simplify(H3t).subs(sympy.exp(-2*I*wd*t),0)

In [133]:
sympy.simplify(H3t).subs([(sympy.exp(2*I*wd*t),0), (sympy.oo,0)])


Matrix([
[                0,                 zoo,                zoo,                  0,                    0,                    0,                0,                0,                0,                0],
[ 4.99334164448514,    5156.98568041008,                  0,                zoo,                  zoo,                  zoo,                0,                0,                0,                0],
[0.257951975085346,                   0,   5284.94056072468,                zoo,                  zoo,                  zoo,                0,                0,                0,                0],
[                0, -0.0167909850402803,  0.205920111856128,    10242.929874963,                    0,                    0,              zoo,              zoo,              zoo,              zoo],
[                0,  -0.470187676635091,   4.97583307954979,                  0,     10441.8481628593,                    0,              zoo,              zoo,              zoo,              zoo],
[